# Cognitive Age Prediction with Structural and Functional MRI

In [17]:
import joblib

## Structural MRI

In [ ]:
# prepare freesurfer data

## Functional MRI

In [22]:
# load connectivity matrices
ATLASES = ['msdl', 'basc064', 'basc122', 'basc197']
# path for the different kind of connectivity matrices
CONNECTIVITY_KIND = ['correlation', 'partial correlation', 'tangent']

sel_atlas = ATLASES[1]
sel_connect = CONNECTIVITY_KIND[0]
sel_subject = 'sub-CC510355'

file_path = f'../../data/connectivity/{sel_subject}/{sel_atlas}/{sel_connect}/'\
            f'{sel_subject}_task-Rest_confounds.pkl'

In [23]:
with open(file_path, 'rb') as f:
    connect_matrix = joblib.load(f)

In [25]:
connect_matrix

array([0.80780958, 0.62181548, 0.69468642, ..., 0.58771813, 0.59074507,
       0.69387215])

scipy stats for Fichers transform
https://en.wikipedia.org/wiki/Fisher_transformation

This is just a tanh function

In [28]:
from itertools import product
meas = ('thickness', 'area')
hemi = ('lh', 'rh')

for h, m in product(hemi, meas):
    print(h, m)

lh thickness
lh area
rh thickness
rh area


In [1]:
from camcan import utils

In [1]:
from camcan.preprocessing import get_structural_data
# test functions on MRI data
subjects_dir = '/home/okozyn/Projects/inria/camcan_analysis/data/freesurfer'
subject = 'CC620466'
out_dir = '/home/okozyn/Projects/inria/camcan_analysis/data/debug'

out_files = get_structural_data(subjects_dir, subject, out_dir)

INFO:root:nsubjects = 1
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/data/tmp.mris_preproc.28847
/home/okozyn/Projects/inria/camcan_analysis/notebook/mri
Log file is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/data/lh.thickness.mris_preproc.log
gio 28 feb 2019, 14.50.56, CET
setenv SUBJECTS_DIR /home/okozyn/Projects/inria/camcan_analysis/data/freesurfer
cd /home/okozyn/Projects/inria/camcan_analysis/notebook/mri
/home/okozyn/Programs/freesurfer/bin/mris_preproc --s CC620466 --target fsaverage5 --hemi lh --meas thickness --out /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/data/lh.thickness.mgh

Linux azzaro 4.15.0-45-generic #48-Ubuntu SMP Tue Jan 29 16:28:13 UTC 2019 x86_64 x86_64 x86_64 GNU/Linux
$Id: mris_preproc,v 1.76.2.1 2016/08/12 17:55:45 zkaufman Exp $
freesurfer-Linux-centos6_x86_64-stable-pub-v6.0.0-2beb96c
tmpdir is /home/okozyn/Projects/inria/camcan_analysis/data/debug/CC620466/data/tmp.mris_preproc.28847
Sr

Exception: Non zero return code: 1. Bash: b''

In [35]:
import nibabel as nb
import numpy as np

img = nb.load(out_files['lh_thickness_file'])
in_data = img.get_data().squeeze()
# column vector
vectorized_data = in_data[:, np.newaxis]

In [36]:
t = np.genfromtxt(out_files['aseg_file'], skip_header=1, delimiter=',')[1:]

In [37]:
out_files.keys()

dict_keys(['lh_thickness_file', 'lh_area_file', 'rh_thickness_file', 'rh_area_file', 'aseg_file'])

In [38]:
t.shape

(66,)

In [33]:
vectorized_data.sort()
np.sum(vectorized_data > 0)

149600

In [39]:
img.get_data().shape

(163842, 1, 1)

In [ ]:
d = [d for d in os.listdir('./') if isdir(d) and 'CC' in d]  